In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2

In [2]:

path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB052570,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325183,Mrs,Carmel,D'Arcy,Female,...,2025-08-31 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
1,KK078746,Karma Resorts,Other,Past Guests,Karma Kandara,PF326128,Mr.,Leroy Sheldon,Haines,Male,...,2025-09-01 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,4,4,1,Email and Phone,NO
2,KK077762,Karma Resorts,Other,Past Guests,Karma Kandara,PF323451,Miss,Celeste Anne,Tallowwood,Female,...,2025-09-05 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,4,1,0,Email and Phone,NO
3,KB052572,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325193,Mr,Daniel,O'Halloran,Male,...,2025-09-02 00:00:00,2,WYNDHAM,WYNDHAM,WYNDHAM,1,0,0,Email and Phone,NO
4,KK079193,Karma Resorts,Other,Past Guests,Karma Kandara,PF327639,Miss,Tara Jane,Matzen,Female,...,2025-09-01 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,KSH025709,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF317021,Ms,Chrissy,Quinn,Female,...,2025-08-31 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email Only,NO
121,KLM006481,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF284434,Mr,Andrew,Neil,Male,...,2025-08-31 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Phone Only,NO
122,KSM016666,Karma Resorts,Other,Past Guests,Karma St Martins,PF325341,<NA>,Anthony,Gibbins,Unknown,...,2025-09-02 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
123,KSM014423,Karma Resorts,Other,Past Guests,Karma St Martins,PF296201,Mrs,Turner Ref Premier,Hol,Unknown,...,2025-09-04 00:00:00,2,TA,TA,RETAIL,2,0,0,Phone Only,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
df_analytic.loc[
    (df_analytic['TelNo'] == 'empty') & (df_analytic['MobileNo'] != 'empty'), 
    'TelNo'] = df_analytic['MobileNo']
df_analytic.loc[df_analytic['TelNo'] == df_analytic['MobileNo'],'MobileNo'] = 'empty'

In [8]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
35,KB048829,Karma Resorts,Other,Past Guests,Karma Bavaria,PF307225,Mrs,Shimul,Shukla,Female,...,2025-09-01 21:30:00,0,CB,TEL,RETAIL,1,0,0,Email and Phone,NO
36,KB048936,Karma Resorts,Other,Past Guests,Karma Bavaria,PF307225,Mrs,Shimul,Shukla,Female,...,2025-09-01 21:30:00,0,CB,TEL,RETAIL,1,0,0,Email and Phone,NO
37,KB048803,Karma Resorts,Other,Past Guests,Karma Bavaria,PF307225,Mrs,Shimul,Shukla,Female,...,2025-09-03 18:00:00,0,CB,TEL,RETAIL,1,0,0,Email and Phone,NO
38,KB048802,Karma Resorts,Other,Past Guests,Karma Bavaria,PF307225,Mrs,Shimul,Shukla,Female,...,2025-09-02 18:00:00,0,CB,TEL,RETAIL,1,0,0,Email and Phone,NO
70,KB045005,Karma Resorts,Other,Past Guests,Karma Bavaria,PF289159,empty,Lee,Hobeika,Unknown,...,2025-08-31 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
72,KSM015441,Karma Resorts,Other,Past Guests,Karma St Martins,PF098426,Mr,Steve,Backshall,Unknown,...,2025-09-01 00:00:00,4,COMP,GROUPAMB,KC,2,0,3,Email and Phone,NO
77,KSM015802,Karma Resorts,Other,Past Guests,Karma St Martins,PF316715,Mrs,Karen,Hounsell,Unknown,...,2025-09-05 00:00:00,1,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
82,KSM015770,Karma Resorts,Other,Past Guests,Karma St Martins,PF316284,Mrs,Alison,Crowter,Unknown,...,2025-09-03 00:00:00,1,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
84,KSM016821,Karma Resorts,Other,Past Guests,Karma St Martins,PF326919,Mrs,Brenda,Lloyd,Unknown,...,2025-09-01 00:00:00,4,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
89,KSM015831,Karma Resorts,Other,Past Guests,Karma St Martins,PF317151,Mr,David,Spencer,Unknown,...,2025-09-04 00:00:00,3,DIR,GHPA,DIGITAL,2,0,0,Email and Phone,NO


In [9]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB052570,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325183,Mrs,Carmel,D'Arcy,Female,...,2025-08-31 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
1,KK078746,Karma Resorts,Other,Past Guests,Karma Kandara,PF326128,Mr.,Leroy Sheldon,Haines,Male,...,2025-09-01 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,4,4,1,Email and Phone,NO
2,KK077762,Karma Resorts,Other,Past Guests,Karma Kandara,PF323451,Miss,Celeste Anne,Tallowwood,Female,...,2025-09-05 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,4,1,0,Email and Phone,NO
3,KB052572,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325193,Mr,Daniel,O'Halloran,Male,...,2025-09-02 00:00:00,2,WYNDHAM,WYNDHAM,WYNDHAM,1,0,0,Email and Phone,NO
4,KK079193,Karma Resorts,Other,Past Guests,Karma Kandara,PF327639,Miss,Tara Jane,Matzen,Female,...,2025-09-01 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,KSH025709,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF317021,Ms,Chrissy,Quinn,Female,...,2025-08-31 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email Only,NO
109,KLM006481,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF284434,Mr,Andrew,Neil,Male,...,2025-08-31 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Phone Only,NO
110,KSM016666,Karma Resorts,Other,Past Guests,Karma St Martins,PF325341,empty,Anthony,Gibbins,Unknown,...,2025-09-02 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
111,KSM014423,Karma Resorts,Other,Past Guests,Karma St Martins,PF296201,Mrs,Turner Ref Premier,Hol,Unknown,...,2025-09-04 00:00:00,2,TA,TA,RETAIL,2,0,0,Phone Only,NO


In [10]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [11]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [12]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB052570,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325183,Mrs,Carmel,D'Arcy,Female,...,2025-08-31 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
3,KB052572,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325193,Mr,Daniel,O'Halloran,Male,...,2025-09-02 00:00:00,2,WYNDHAM,WYNDHAM,WYNDHAM,1,0,0,Email and Phone,NO
18,KB048276,Karma Resorts,Other,Past Guests,Karma Bavaria,PF178352,Mrs,Wei,Yu,Female,...,2025-08-31 00:00:00,2,COMP,GER_OWE,COMP,2,1,0,Email and Phone,NO
20,KK068860,Karma Resorts,Other,Past Guests,Karma Kandara,PF299103,Miss,Caroline,Impler,Female,...,2025-09-06 00:00:00,1,HOUSE_USE,GER_OWE,COMP,3,0,0,Email and Phone,NO
61,KK069756,Karma Resorts,Other,Past Guests,Karma Kandara,PF077530,Mrs,Irina,Shutova,Female,...,2025-09-05 00:00:00,16,HOUSE_USE,V_OWNER,COMP,1,0,0,Email and Phone,NO
73,KSM016885,Karma Resorts,Other,Past Guests,Karma St Martins,PF164907,Miss,Andrea,Hawken,Unknown,...,2025-09-02 00:00:00,4,DIR,TEL,RETAIL,1,0,0,Email and Phone,NO
75,KSM016827,Karma Resorts,Other,Past Guests,Karma St Martins,PF326919,Mrs,Brenda,Lloyd,Unknown,...,2025-09-02 00:00:00,1,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO


In [13]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [14]:
count_data_akhir = df_analytic.shape[0]

In [15]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KSM016193,Karma Resorts,Other,Past Guests,Karma St Martins,PF319998,empty,Miles Westbrook,Lunn,Unknown,...,2025-08-31 00:00:00,1,OTA,OTA,RETAIL,1,0,0,Phone Only,NO
1,KB045004,Karma Resorts,Other,Past Guests,Karma Bavaria,PF289159,empty,Lee,Hobeika,Unknown,...,2025-08-31 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
2,KSH025710,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF317031,Ms,Shannon,Mckee,Female,...,2025-08-31 00:00:00,1,DIGITAL,WEDDING,GROUP,2,0,0,Email Only,NO
3,KLM008248,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF303079,Mr,Robbie,Macmurchie,Male,...,2025-08-31 00:00:00,1,OTA,WEDDING,GROUP,2,0,0,Email and Phone,NO
4,KB050428,Karma Resorts,Other,Past Guests,Karma Bavaria,PF328173,Frau,Victoria,Schiller,Female,...,2025-08-31 00:00:00,1,LGR,LGR,GROUP,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,KB046433,Karma Resorts,Other,Past Guests,Karma Bavaria,PF252952,Herr,Hans Jürgen,Jistel,Male,...,2025-09-05 21:00:00,0,LGR,TEL,RETAIL,1,0,0,Phone Only,NO
102,KB047931,Karma Resorts,Other,Past Guests,Karma Bavaria,PF303465,empty,Martina,Schmidt,Unknown,...,2025-09-06 00:00:00,1,OTA,OTA,RETAIL,6,0,0,Phone Only,NO
103,KSM014769,Karma Resorts,Other,Past Guests,Karma St Martins,PF303742,Mrs,Tba,Barker,Unknown,...,2025-09-06 00:00:00,2,TA,TEL,RETAIL,2,0,0,Email and Phone,NO
104,KB052228,Karma Resorts,Other,Past Guests,Karma Bavaria,PF323434,empty,Dr. Tim,Veil,Unknown,...,2025-09-07 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO


In [16]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [17]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_21668\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [18]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KSM016193,Karma Resorts,Other,Past Guests,Karma St Martins,PF319998,,Miles Westbrook,Lunn,Unknown,...,2025-08-31,1,OTA,OTA,RETAIL,1,0,0,Phone Only,NO
1,KK065181,Karma Resorts,Other,Past Guests,Karma Kandara,PF295051,Mr,Christopher,Tran,Male,...,2025-08-31,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
2,KLM005255,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF269973,Mrs,Tracey,Smith,Female,...,2025-08-31,2,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
3,KB050431,Karma Resorts,Other,Past Guests,Karma Bavaria,PF328220,Mr,Alexis,Rousset-Monredon,Male,...,2025-08-31,2,LGR,LGR,GROUP,4,0,0,Email and Phone,NO
4,KB050431,Karma Resorts,Other,Past Guests,Karma Bavaria,PF328200,Mr,Adrien,Coulaud,Male,...,2025-08-31,2,LGR,LGR,GROUP,4,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,KSM016932,Karma Resorts,Other,Past Guests,Karma St Martins,PF317151,Mr,David,Spencer,Unknown,...,2025-09-05,1,DIR,WALKIN,RETAIL,2,0,0,Email and Phone,NO
102,KB047931,Karma Resorts,Other,Past Guests,Karma Bavaria,PF303465,,Martina,Schmidt,Unknown,...,2025-09-06,1,OTA,OTA,RETAIL,6,0,0,Phone Only,NO
103,KSM014769,Karma Resorts,Other,Past Guests,Karma St Martins,PF303742,Mrs,Tba,Barker,Unknown,...,2025-09-06,2,TA,TEL,RETAIL,2,0,0,Email and Phone,NO
104,KB052228,Karma Resorts,Other,Past Guests,Karma Bavaria,PF323434,,Dr. Tim,Veil,Unknown,...,2025-09-07,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO


In [19]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 125
Jumlah Data Yang Duplicate : 12
Jumlah Data Setelah Hapus Duplicate : 113
Jumlah Data Yang Termasuk Member: 7
Jumlah Data Setelah Hapus Member : 106
Jumlah Data Akhir : 106


In [20]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)